# ALL CITATIONS

## Acquire
Data: [Los Angeles Parking Citations](https://www.kaggle.com/cityofLA/los-angeles-parking-citations)<br>
Load the dataset and filter for:
- Citations issued from 2017-01-01 to 2020-12-22.
- Street Sweeping violations - `Violation Description` == __"NO PARK/STREET CLEAN"__

Let's acquire the parking citations data from our file.
1. Import libraries.
1. Load the dataset.
1. Display the shape and first/last 2 rows.
1. Display general infomation about the dataset - w/ the # of unique values in each column.
1. Display the number of missing values in each column.
1. Descriptive statistics for all numeric features.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import sys
import time

# Insert path to the source folder.
sys.path.insert(1, 'src/')
import acquire
import prepare

# Filter warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# Load the data
df = pd.read_csv('parking-citations.csv')

In [3]:
# Display the shape and dtypes of each column
print(df.shape)
df.info()

(12694615, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12694615 entries, 0 to 12694614
Data columns (total 22 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Ticket number           object 
 1   Issue Date              object 
 2   Issue time              float64
 3   Meter Id                object 
 4   Marked Time             float64
 5   RP State Plate          object 
 6   Plate Expiry Date       float64
 7   VIN                     object 
 8   Make                    object 
 9   Body Style              object 
 10  Color                   object 
 11  Location                object 
 12  Route                   object 
 13  Agency                  float64
 14  Violation code          object 
 15  Violation Description   object 
 16  Fine amount             float64
 17  Latitude                float64
 18  Longitude               float64
 19  Agency Description      object 
 20  Color Description       object 
 21  Body Style Des

In [4]:
# Display the first two citations
df.head(2)

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,...,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Agency Description,Color Description,Body Style Description
0,1103341116,2015-12-21T00:00:00.000,1251.0,NaN,NaN,CA,200304.0,NaN,HOND,PA,...,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,NaN,NaN,NaN
1,1103700150,2015-12-21T00:00:00.000,1435.0,NaN,NaN,CA,201512.0,NaN,GMC,VN,...,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,NaN,NaN,NaN


In [5]:
# Display the last two citations
df.tail(2)

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,...,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Agency Description,Color Description,Body Style Description
12694613,1111677140,2021-03-17T00:00:00.000,2055.0,NaN,NaN,CA,202110.0,NaN,NISS,PA,...,00314,1.0,22500H,NaN,68.0,6.463469e+06,1.833881e+06,WESTERN,GREY,PASSENGER CAR
12694614,1114412003,2021-03-11T00:00:00.000,1932.0,NaN,NaN,CA,202109.0,NaN,FORD,PA,...,3Z2,1.0,8056E4,NaN,93.0,9.999900e+04,9.999900e+04,WESTERN,NaN,PASSENGER CAR


In [6]:
# Display descriptive statistics of numeric columns
df.describe()

,Issue time,Marked Time,Plate Expiry Date,Agency,Fine amount,Latitude,Longitude
count,1.268815e+07,393731.000000,1.146102e+07,1.269135e+07,1.262809e+07,1.269461e+07,1.269461e+07
mean,1.191180e+03,1062.090483,1.898542e+05,5.196745e+01,7.092119e+01,5.601162e+06,1.613639e+06
std,4.722410e+02,223.820934,4.759132e+04,9.467209e+00,3.525635e+01,2.713570e+06,1.737902e+06
min,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,9.070000e+02,916.000000,2.016060e+05,5.100000e+01,6.300000e+01,6.423183e+06,1.823486e+06
50%,1.148000e+03,1041.000000,2.018010e+05,5.400000e+01,6.800000e+01,6.453856e+06,1.842128e+06
75%,1.455000e+03,1202.000000,2.019080e+05,5.500000e+01,7.300000e+01,6.475488e+06,1.858207e+06
max,2.359000e+03,2400.000000,9.201020e+05,9.900000e+01,1.100000e+03,4.042322e+09,4.042322e+09


__Initial findings__
- `Issue time` and `Marked Time` are quasi-normally distributed. Note: Poisson Distribution
- It's interesting to see the distribution of our activity on earth follows a normal distribution.
- Agencies 50+ write the most parking citations.
- Most fine amounts are less than $100.00
- There are a few null or invalid license plates.

# Prepare

- Remove spaces + capitalization from each column name.
- Cast `Plate Expiry Date` to datetime data type.
- Cast `Issue Date` and `Issue Time` to datetime data types.
- Drop columns missing >=74.42\% of their values. 
- Drop missing values.
- Transform Latitude and Longitude columns from NAD1983StatePlaneCaliforniaVFIPS0405 feet projection to EPSG:4326 World Geodetic System 1984: used in GPS [Standard]
- Filter data for street sweeping citations only.

In [7]:
formatted_column_names = [x.replace(' ', '_').lower() for x in df.columns.to_list()]
df.columns = formatted_column_names

# Cast issue_date and issue_time from a string to a datetime data type.
df.issue_date = pd.to_datetime(df.issue_date)
df.issue_time = pd.to_datetime(df.issue_time, format='%H%M', errors='coerce')

# Drop columns, convert coordinates, and add new features
df = prepare.drop_features(df)
df = prepare.convert_coordinates(df)
df = prepare.add_features(df)

df = df[(df.issue_date >= '2016-01-01')&(df.issue_date <= '04-12-2021')]

# Drop the index and sort by issue_date
df = df.sort_values(by=['issue_date', 'issue_time'])
df.reset_index(drop=True, inplace=True)

# Cache file
df.to_csv('prepared-parking-citations.csv', index=False)

In [8]:
df.head()

,issue_date,issue_time,rp_state_plate,plate_expiry_date,make,body_style,color,location,route,agency,violation_description,fine_amount,latitude,longitude,day_of_week,issue_year,issue_hour,issue_minute
0,2016-01-01,1900-01-01 01:00:00,CA,201509.0,HOND,PA,RE,VANALDEN/ENADIA,10A17,1.0,NO EVIDENCE OF REG,50.0,27.5175,-137.9131,Friday,2016,1,0
1,2016-01-01,1900-01-01 01:02:00,CA,201605.0,VOLK,PA,BK,1735 CHRISLER,6Q31,1.0,22514,68.0,27.5175,-137.9131,Friday,2016,1,2
2,2016-01-01,1900-01-01 01:06:00,CA,201604.0,FORD,PA,GY,N/W 6TH ST/OXFORD,20A33,1.0,RED ZONE,93.0,27.5175,-137.9131,Friday,2016,1,6
3,2016-01-01,1900-01-01 02:50:00,CA,201609.0,HOND,PA,GY,400 S WESTMINSTER AVE,00403,54.0,DISPLAY OF PLATES,25.0,34.0671,-118.3162,Friday,2016,2,50
4,2016-01-01,1900-01-01 02:52:00,CA,201609.0,HOND,PA,BK,400 S WESTMINSTER AVE,00403,54.0,DISPLAY OF PLATES,25.0,34.0671,-118.3162,Friday,2016,2,52
